# **Binary classification**
## Tabular data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures # 좋은 feature들끼리 곱하면 더 좋은 feature를 만들수도 있음.! 그때 곱하기를 하려면 'PolynomialFeatures'가 필요
from sklearn.preprocessing import StandardScaler


# VarianceThreshold : 좋은 feature를 찾기 위해 사용
# variance가 작다는 말은 정보가 적다는 말과 비슷
# 다양하게 스펙트럼이 넓어야 얻을 수 있는 정보가 있을 텐데  
# example : 1개의 feature와 그 안에 100개의 sample(여기서 sample은 [0 or 1]의 값을 가진다고 하면)이 있다고 할때, sample의 구성이 1이 99개고 0이 1개다. 
# --> variance가 거의 없고 얻을 정보가 없는 feature라고 판단됨 --> 그런 feature를 제거하기 위해 'VarianceThreshold'를 사용!

from sklearn.feature_selection import VarianceThreshold


# SelectFromModel : model마다 학습을 할 때 Feature의 중요도를 결정하게 되는데 그 중요도에 따라서 selection하는 것.
# 결국 모델을 하나씩 학습을 해야 함. 만약 feature가 1000개면 1000개의 모델을 만들어야 한다는 뜻 
from sklearn.feature_selection import SelectFromModel


from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100) # 최대 100개까지 열의 수를 보여주도록



# from sklearn.preprocessing import Imputer 

# ---->

# 해결 방법
# Imputer 3 버전 전에 사용되지 않으며 0.22에서 제거되었다. 
# Imputer 모듈을 불러오기 위해선 sklearn.impute를 사용하면 된다.
# Imputer : 비어져있는 null value를 채우는 sklearn 내장 함수

from sklearn.impute import SimpleImputer

> DEBUGGING
- DEBUGGING 하는데 굳이 전체 dataset을 다 읽어서 할 이유는 없다..
- DEBUGGING을 통해 model을 만들기, ensemble,..등의 과정을 checking 해보고 잘 되면 
- DEBUG = False 를 통해 다시 전체 dataset을 다 읽어서 학습을 진행하면 됨

##### There are only about 600,000 train data for this competition, so I'll just read them all!

In [ ]:
# DEBUG = True
DEBUG = False
# debugging 마무리 후 
# DEBUG = False 사용해서 학습 마무리 하기

In [ ]:
if DEBUG:
    NROWS = 50000
else:
    NROWS = None

In [ ]:
%time

train = pd.read_csv('../input/porto-seguro-safe-driver-prediction/train.csv', nrows = NROWS)
test = pd.read_csv('../input/porto-seguro-safe-driver-prediction/test.csv', nrows = NROWS)


##--------------------train set이 너무 클때!!-------------------------##

# train = train.sample(frac=0.2)
# train data의 20%를 sample로 가져오겠다.
# 주로 data의 크기가 너무 클때 ex)train data가 80,000,00개 정도 있다고 하면 
# 간단한 EDA정도와 feature engineering 진행할 때는 sample로 10~20%정도 (본인 컴퓨터 사양에 따라) sampling을 해서 진행하면 수월하다.! (여기서 sampling은 random_sampling)
# 전체 set을 가지고 EDA를 하는것은 무모하고..쉽지 않다

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.tail()

> we can see 'categorical variables of which the category values are integers! let's check it out

In [ ]:
cat_cols = [col for col in train.columns if 'cat' in col]

In [ ]:
# for col in cat_cols:
#     print(col, train[col].value_counts().shape[0])   #여기서 shape[0]은 각 col의 unique한 행의 수를 가져오기 위함

for col in cat_cols:
    print(col, train[col].nunique())   # 'nunique'를 사용해도 됨

> we can see 'high cardinality' in the 'ps_car_11_cat' category! **104 !!**

> let's see if there are duplicate rows in the training data. Using 'drop_duplicates()'


In [ ]:
train.drop_duplicates()
train.shape

- no duplicate rows, so that's fine

In [ ]:
test.shape

>  let's investigate how many variables of each type we have! 

> So later on we can create dummy variables for the 14 categorical variables. The bin variables are already binary and do not need dummification.! **(dummification == one-hot encoding)**

#### Dummy variable

1. what is Dummy variable
- categorical variable -> continuous variable
- 범주형 변수를 연속형 변수로 변환. 즉 연속형 변수'스럽게' 만드는 것

2. Why we should make Dummy variable
- 범주형 변수로는 사용할 수 없고 연속형 변수로만 가능한 분석기법을 사용할 수 있게 해줌.
- linear regression, logistic regression : 설명변수가 연속형 변수여야 사용 가능.
- 만약, 설명변수 중 범주형 변수가 있다면, 그 변수를 더미 변수로 변환해 분석에 사용

3. The features of Dummy variable
- 0또는1의 값을 가진다
- 더미변수는 원래 범주형 변수의 범주 개수보다 1개 적게 만들어진다(더미 변수로 만들어지지 않고 생략되는 범주는 기준이 되는 값이라고 이해하면 됨)

4. Make Dummy variable
- 범주형 변수의 범주 중 기준이 되는 값을 정한다
- 기준이 되는 변수를 제외하고 더미변수를 만든다

5. The meaning of Dummy variable
- 회귀식에서 해당 변수의 효과를 0또는 상수값으로 만들어 준다
- 더미변수는 회귀 기울기를 바꾸지는 않고 절편만 바꾸어 평행하게 움직이게 하는 역할을 한다

In [ ]:
train.info()